In [1]:
import requests
import json
import os
import time
import random
import sys
import pandas as pd
sys.path.append("..")
import sageCommons
apiKey=os.getenv('CKANAPIKEY')
if apiKey is None:
    raise EnvironmentError("Failed because {} is not set.".format('CKANAPIKEY'))

In [2]:
mainDir = '/Users/iperezx/Documents/sage-commons/sage-commons/'
#sageCommonsURL='http://127.0.0.1:5000'
sageCommonsURL='http://hotshot.sdsc.edu:5000'
orgName= 'sage'

In [3]:
commons = sageCommons.sageCommons(sageCommonsURL,apiKey)

## create datasets for sensors

In [4]:
sensorMetadataFile = 'sensors.json'
sensorMetadata = json.load(open(sensorMetadataFile))
# print(dataJSON)
sensorMetadataDF = pd.DataFrame.from_dict(sensorMetadata, orient='columns')

In [5]:
# get sensor hardware
sensorHardwareURL = 'https://sage-data-api.nautilus.optiputer.net/api/v1/sensor-hardware-data'

response = requests.request("GET", sensorHardwareURL)
sensorHardwareJSON = response.json()
# print(sensorHardwareJSON['data'])

sensorHardwareDF = pd.DataFrame.from_dict(sensorHardwareJSON['data'], orient='columns')
sensorHardwareDF.rename(columns={'product_name':'name'},inplace=True)
sensorHardwareDF

,id,name,manufacture,sensor_types,link
0,000000000,bme680,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
1,000000001,rg-15,Hydreon Corporation,rainfall measuring device,https://rainsensors.com/products/rg-15/
2,000000002,bme280,Bosch Sensortec,"relative humidity,barometric pressure,ambient ...",https://www.bosch-sensortec.com/products/envir...
3,000000003,xnv-8081z,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8081z.html
4,000000004,xnf-8010rv,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnf-8010rv.html
5,000000005,xnv-8080r,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8080r.html
6,000000006,xnv-8082r,Hanwha Techwin America,camera,https://www.hanwhasecurity.com/xnv-8082r.html


In [6]:
colmnNames = ['name','sensor_types']
combinedData = sensorMetadataDF.merge(sensorHardwareDF[colmnNames], how='inner', on='name')
combinedData

,name,title,url,notes,tags,resource,sensor_types
0,xnf-8010rv,X series 6 MP sensor 360˚ Outdoor Fisheye: xnf...,https://www.hanwhasecurity.com/xnf-8010rv.html,Wisenet X powered by Wisenet 5 network outdoor...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...",camera
1,xnv-8080r,5MP Vandal IR Dome Camera: xnv-8080r,https://www.hanwhasecurity.com/xnv-8080r.html,Wisenet X powered by Wisenet 5 network IR outd...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...",camera
2,xnv-8082r,6MP Vandal-Resistant IR Outdoor Network Dome C...,https://www.hanwhasecurity.com/xnv-8082r.html,The Wisenet X series powered by Wisenet 7 netw...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...",camera
3,xnv-8081z,5MP Vandal-Resistant Outdoor Network Dome PTRZ...,https://www.hanwhasecurity.com/xnv-8081z.html,The Wisenet X series PLUS powered by Wisenet 5...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...",camera
4,rg-15,Rainfall Sensor: rg-15,https://rainsensors.com/products/rg-15/,The Hydreon RG-15 Solid State Tipping Bucket i...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...",rainfall measuring device
5,bme680,"High-accuracy gas, pressure, humidity and temp...",https://www.bosch-sensortec.com/products/envir...,The BME680 is the first gas sensor that integr...,[sensor],"[{'name': 'Datasheet', 'format': 'PDF', 'url':...","relative humidity,barometric pressure,ambient ..."


In [7]:
combinedData['tags'] = combinedData.apply(lambda x: x['tags'] + x['sensor_types'].lstrip().split(','),axis=1)
combinedData.drop(columns=['sensor_types'],inplace=True)

In [8]:
sensorMetadataMod = combinedData.to_dict('records')

In [9]:
sensorMetadataMod

[{'name': 'xnf-8010rv',
  'title': 'X series 6 MP sensor 360˚ Outdoor Fisheye: xnf-8010rv',
  'url': 'https://www.hanwhasecurity.com/xnf-8010rv.html',
  'notes': 'Wisenet X powered by Wisenet 5 network outdoor vandal fisheye, 6 MP CMOS sensor, 2048x2048 @ 30fps with WDR off/on(120dB), 1.6mm fixed lens (192°), H.265/H.264/MJPEG, WiseStream II compression technology, USB port for easy installation, advanced video/audio and business analytics, simple focus, dual SD card slots, HLC, Defog detection, DIS , 12VDC/PoE, IP66, IK10\r\n\r\nKey features:\r\n\r\n* 2048 x 2048 resolution, 30fps\r\n* Fisheye lens 1.6mm (192°X192°) W/ simple focus\r\n* Built-in IR for up to 49’ or 15m\r\n* Triple Codec: H.265, H.264 and MJPEG multiple streaming\r\n* WiseStream II compression technology\r\n* WDR 120dB @30fps\r\n* Loitering, directional detection, fog detection, tampering, Motion detection, object enter or exit an area\r\n* Sound Classification, heatmap, people counting and queue line management\r\n* D

In [10]:
commons.uploadDataForEntity(orgName,sensorMetadataMod)

### create data resource - test

In [ ]:
# resources = dataJSON[0]
# datasetID,response = commons.uploadDataset(orgName,resources)
# jsonResponseDS = response.json()
# print(response)
# print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

### create data source - test

In [ ]:
# datasetID
# resource = dataJSON[0]['resource'][0]
# print(resource)
# response = commons.uploadResource(datasetID,resource)
# jsonResponse = response.json()
# print(json.dumps(jsonResponse, indent=4, sort_keys=True))